#### Imports

In [ ]:
import os
import re
import json

import numpy as np
%load_ext cudf.pandas
import pandas as pd
import networkx as nx
import plotly.express as px

from itertools import chain
from unidecode import unidecode
from matplotlib import pyplot as plt

#### Configurating default behavior

In [ ]:
pd.set_option('display.max_rows', 10)
pd.set_option('display.max_columns', 50)
pd.set_option('display.width', 15)

In [ ]:
files_path = './data/universidade_federal_do_rio_grande_do_norte_-_ufrn'

In [ ]:
json_path = './data/universidade_federal_do_rio_grande_do_norte_-_ufrn.json'

In [ ]:
with open(json_path, 'r') as f:
    json_data = json.load(f)

#### Auxiliar functions

In [ ]:
online_resources = list(chain(*[[l['link'] for l in d['recursos'] if 'formato' in l and l['formato'] == 'CSV'] for d in json_data['conjuntoDados']]))
offline_resources = [os.path.join(files_path,p) for p in os.listdir(files_path)]

In [ ]:
db_resources = online_resources
db_resources = offline_resources

In [ ]:
resources_containing = lambda sw: [r for r in db_resources if sw in r and r.endswith('.csv')]
dataframe_containing = lambda sw : pd.concat([pd.read_csv(csv, sep=';',decimal=',') for csv in resources_containing(sw)])

#### Loading curricular data

In [ ]:
compontentes = dataframe_containing("componentes-curriculares")

In [ ]:
estruturas_curriculares = dataframe_containing('estruturas-curriculares')

In [ ]:
estruturas_curriculares_componentes = dataframe_containing('curriculo-componente')

https://sparkbyexamples.com/pandas/pandas-group-dataframe-rows-list-groupby/

In [ ]:
matriculas = []

for matricula_csv_filepath in resources_containing('matricula'):
    matriculas.append(pd.read_csv(
        matricula_csv_filepath, 
        sep=';',
        decimal=',',
        usecols=[
            'id_turma',
            'descricao',
            'discente',
            'unidade',
            'nota'
        ],dtype={
            'id_turma':int,
            'descricao':str,
            'discente':str,
            'unidade':float,
            'nota':float
        }).drop_duplicates().sort_values([
            'id_turma',
            'descricao',
            'discente',
            'unidade',
            'nota'
        ]).groupby([
            'id_turma',
            'discente',
            'descricao'
        ]).agg(list).reset_index()
    )
    
matriculas = pd.concat(matriculas)

In [ ]:
matriculas['n_unidade'] = matriculas['nota'].apply(len)

In [ ]:
matriculas[==6]

In [ ]:
px.histogram(matriculas['nota'].apply(np.nanmean),nbins=100)

In [ ]:
y = df_gb['nota'].apply(list)

#### Filtering curricular data

In [ ]:
id_valid_year = estruturas_curriculares['ano_entrada_vigor']!=0

id_estruturas_curriculares = id_valid_year

estruturas_curriculares = estruturas_curriculares[id_estruturas_curriculares]

In [ ]:
id_valid_curricula = estruturas_curriculares_componentes['id_curriculo'].isin(estruturas_curriculares['id_curriculo'])
id_valid_component = estruturas_curriculares_componentes['id_componente_curricular'].isin(compontentes['id_componente'])

id_estruturas_curriculares_componentes = id_valid_curricula & id_valid_component

estruturas_curriculares_componentes = estruturas_curriculares_componentes[id_estruturas_curriculares_componentes]

In [ ]:
id_graduacao = compontentes['nivel'] == "G"

id_componentes = np.array([True]*len(compontentes))
# id_componentes = id_componentes & id_graduacao))))

compontentes = compontentes[id_componentes]

#### Transforming data

##### Requirement tree

In [ ]:
compontentes['equivalencia'] = compontentes['equivalencia'].apply(lambda x: x.strip(' ') if type(x) == str else '(  )')
compontentes['co_requisito'] = compontentes['co_requisito'].apply(lambda x: x.strip(' ') if type(x) == str else '(  )')
compontentes['pre_requisito'] = compontentes['pre_requisito'].apply(lambda x: x.strip(' ') if type(x) == str else '(  )')

In [ ]:
id_component_invalid = compontentes['pre_requisito'].apply(lambda x: len(re.findall(r'\b[^\.](\d+)',x)) > 0)

In [ ]:
for x in compontentes[id_component_invalid].iloc:
    id_componentes = [int(x) for x in x['pre_requisito'].strip('( )').split()]
    comp = compontentes[compontentes['id_componente'].isin(id_componentes)]
    
    pre_requisito = '( ( ' + ' ) E ( '.join(comp['codigo'].values) + ' ) )'
    
    compontentes.loc[compontentes['id_componente'] == x['id_componente'],'pre_requisito'] = pre_requisito

### Filtering

In [ ]:
all_compontentes = compontentes.copy()
all_estruturas_curriculares = estruturas_curriculares.copy()
all_estruturas_curriculares_componentes = estruturas_curriculares_componentes.copy()

In [ ]:
patterns =['cien.*comp','tecno.*info','eng.*sof']
id = [False] * all_estruturas_curriculares.shape[0]
for pattern in patterns:
    id = id | all_estruturas_curriculares['nome_curso'].apply(lambda nome:  len(re.findall(pattern,unidecode(nome.lower())))!=0).values
estruturas_curriculares = all_estruturas_curriculares[id]

In [ ]:
estruturas_curriculares_componentes = all_estruturas_curriculares_componentes[all_estruturas_curriculares_componentes['id_curriculo'].isin(estruturas_curriculares['id_curriculo'])]
estruturas_curriculares = estruturas_curriculares[estruturas_curriculares['id_curriculo'].isin(estruturas_curriculares_componentes['id_curriculo'])]
estruturas_curriculares = estruturas_curriculares[estruturas_curriculares['nome_matriz'].apply(lambda nm: not nm is np.nan)]

## Analysing DIMAP/IMD data

In [ ]:
estrutura_curricular = estruturas_curriculares.iloc[0]
estrutura_curricular

In [ ]:
estrutura_curricular_componentes_info = estruturas_curriculares_componentes[estruturas_curriculares_componentes['id_curriculo'] == estrutura_curricular['id_curriculo']]
estrutura_curricular_componentes_info = estrutura_curricular_componentes_info.merge(compontentes, left_on='id_componente_curricular', right_on='id_componente')
estrutura_curricular_componentes_info = estrutura_curricular_componentes_info.merge(estruturas_curriculares, left_on='id_curriculo', right_on='id_curriculo')

In [ ]:
estrutura_curricular_codigo_componentes = set(estrutura_curricular_componentes_info['codigo_x'].values)

estrutura_curricular_codigo_componentes.update(set(chain(*estrutura_curricular_componentes_info['equivalencia'].apply(lambda x: re.findall(r'\w+\d+',x) if type(x) == str else []).values)))
estrutura_curricular_codigo_componentes.update(set(chain(*estrutura_curricular_componentes_info['co_requisito'].apply(lambda x: re.findall(r'\w+\d+',x) if type(x) == str else []).values)))
estrutura_curricular_codigo_componentes.update(set(chain(*estrutura_curricular_componentes_info['pre_requisito'].apply(lambda x: re.findall(r'\w+\d+',x) if type(x) == str else []).values)))

In [ ]:
estrutura_curricular_componentes = compontentes[compontentes['codigo'].isin(estrutura_curricular_codigo_componentes)]

In [ ]:
estrutura_curricular

In [ ]:
a = estrutura_curricular_componentes_info[estrutura_curricular_componentes_info['tipo_vinculo_componente'] == 'OBRIGATÓRIO']

In [ ]:
a['equivalencia']

In [ ]:
re.sub('(\w+\d+)',r'( \1 )',a['equivalencia'].iloc[0])

In [ ]:
i = 0

In [ ]:
i = 10

In [ ]:
# i+=1
x1 = a['equivalencia'].iloc[i]
x2 = re.sub('[^\(]\s?(\w+\d+)\s?´^\)]', r' ( \1 ) ', x1)

print(i)
print(x1)
print(x2)


In [ ]:
a['equivalencia']

In [ ]:
a['equivalencia'].apply(lambda x: (x,re.sub('\s*\(?\s*(\w+\d+)\s*\)\s*',r' ( \1 ) ',x)))

In [ ]:
[(eq,re.findall('([^\(\)]+)',eq)) for eq in a['equivalencia'].iloc]

In [ ]:
for eq in a['equivalencia'].iloc:
    print(eq,re.findall('[^\( \)]+',eq))
    print(eq,re.findall('([^\(\)]+)',eq))

In [ ]:
a

In [ ]:
id_ecc_wo_equivalence = estrutura_curricular_componentes['equivalencia'].apply(lambda x: x =='(  )')
id_ecc_wo_requirement = estrutura_curricular_componentes['pre_requisito'].apply(lambda x: x =='(  )')
id_ecc_wo_corequirement = estrutura_curricular_componentes['co_requisito'].apply(lambda x: x =='(  )')

id_ecc = id_ecc_wo_equivalence & id_ecc_wo_requirement & id_ecc_wo_corequirement

In [ ]:
compontentes = compontentes[(compontentes['unidade_responsavel'] == 'INSTITUTO METROPOLE DIGITAL') | (compontentes['unidade_responsavel'] == 'DEPARTAMENTO DE INFORMÁTICA E MATEMÁTICA APLICADA')]
compontentes = compontentes[['id_componente','codigo','nome','ch_total']]

In [ ]:
turmas = dataframe_containing("turma")

turmas = turmas[turmas['nivel_ensino'] == 'GRADUAÇÃO']
turmas = turmas[turmas['situacao_turma'] == 'CONSOLIDADA']
turmas = turmas[turmas['id_componente_curricular'].isin(compontentes['id_componente'])]
turmas.data_consolidacao = pd.to_datetime(turmas.data_consolidacao)

turmas = turmas[['id_turma','siape','id_componente_curricular','ch_dedicada_periodo','descricao_horario','data_consolidacao']]
turmas['ano'] = [d.year for d in turmas.data_consolidacao]
turmas = turmas.dropna()

In [ ]:
matriculas = dataframe_containing("matricula")

matriculas.drop(['unidade','nota','reposicao','faltas_unidade'], axis=1, inplace=True)
matriculas = matriculas[matriculas['id_turma'].isin(turmas['id_turma'])]

matricula_id_descricao = { k:v for k,v in enumerate(matriculas['descricao'].unique()) }
matricula_descricao_id = { v:k for k,v in enumerate(matriculas['descricao'].unique()) }

matriculas['descricao'] = matriculas['descricao'].map(matricula_descricao_id)
matriculas = matriculas[matriculas['media_final'] != 0]
matriculas = matriculas.drop_duplicates()

In [ ]:
cursos = dataframe_containing('cursos-ufrn')
cursos = cursos[cursos['id_curso'].isin(matriculas['id_curso'])]
cursos = cursos[['id_curso','nome']]

In [ ]:
import plotly.express as px

In [ ]:
px.histogram(matriculas, x='media_final', nbins=100,width=800)

In [ ]:
turmas

In [ ]:
px.histogram(matriculas, y='descricao',width=800)

In [ ]:
plt.subplots(4,4,figsize=(20,10))
c = 0
for g,gg in turmas.groupby('ano'):
    c+=1
    ax = plt.subplot(4,4,c)
    plt.hist(matriculas[matriculas['id_turma'].isin(gg.id_turma)].media_final,bins=np.arange(0,10.5,.5));
    plt.title(int(g))
    plt.ylim(0,1500)
    plt.xticks(np.arange(0,10.5,1))
    plt.grid(True)
    pos = ax.get_position()
    pos.y1 = pos.y1*.95
    ax.set_position(pos)
    
    

In [ ]:
df = matriculas.merge(turmas, on='id_turma', how='right')
df.rename(columns={'id_componente_curricular':'id_componente'}, inplace=True)
df = df.merge(compontentes, on='id_componente', how='left')
df = df.dropna()
df = df.drop_duplicates()
df['ano'] = df['ano'].astype(int)
df['descricao_horario'] = [' '.join(re.findall('[2-7]+[MTN][1-6]+',h)) for h in df['descricao_horario']]

In [ ]:
df.sample(4).T

In [ ]:
from itertools import product

In [ ]:
graphs_by_year = {}
for ano,df_ano in df.groupby(['ano']):
    Graph1 = nx.Graph()
    for codigo,disciplina in df_ano.groupby('codigo'):
        media = disciplina['media_final'].mean()
        mediana = disciplina['media_final'].median()
        Graph1.add_node(codigo, nome=disciplina['nome'].iloc[0], media=media, mediana=mediana)

    for d,discente in df_ano.groupby('discente'):
        A = discente[['codigo','media_final']].values
        for a,b in product(A,A):
            Graph1.add_edge(a[0],b[0],mean=(a[1]+b[1])/2)
    
    graphs_by_year[ano[0]] = Graph1

In [ ]:
nx.draw(graphs_by_year[2008],with_labels=True)

In [ ]:
from pyvis.network import Network

In [ ]:
nt = Network('100%', '500px',notebook=True,cdn_resources='in_line')
# populates the nodes and edges data structures
nt.from_nx(graphs_by_year[2009])
nt.barnes_hut()

nt.show_buttons(filter_=['physics'])
nt.show('nx.html')

In [ ]:
graphs_by_year[2009].nodes(data=True)

In [ ]:
graphs_by_year[2009].edges(data=True)